In the modular approach I will create separate files for different model types. This one will be for alpha_stack_model_sil

# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid
from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

# Function

## calculate_gini

In [2]:
def calculate_gini(pd_scores, bad_indicators):
    """
    Calculate Gini coefficient from scores and binary indicators
    
    Parameters:
    pd_scores: array-like of scores/probabilities
    bad_indicators: array-like of binary outcomes (0/1)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays and ensure they're numeric
    pd_scores = np.array(pd_scores, dtype=float)
    bad_indicators = np.array(bad_indicators, dtype=int)
    
    # Check for valid input data
    if len(pd_scores) == 0 or len(bad_indicators) == 0:
        return np.nan
    
    # Check if we have both good and bad cases (needed for ROC AUC)
    if len(np.unique(bad_indicators)) < 2:
        return np.nan
    
    # Calculate AUC using sklearn
    try:
        auc = roc_auc_score(bad_indicators, pd_scores)
        # Calculate Gini from AUC
        gini = 2 * auc - 1
        return gini
    except ValueError:
        return np.nan

## calculate_periodic_gini_prod_ver_trench_dimfact

In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta
from itertools import combinations

def calculate_gini(scores, labels):
    """
    Calculate Gini coefficient with proper handling of edge cases
    
    Returns np.nan when:
    - Fewer than 2 observations
    - No positive labels (sum of labels = 0)
    """
    n = len(scores)
    if n < 2:
        return np.nan
    
    label_sum = np.sum(labels)
    
    # Handle case where no positive labels exist (all zeros)
    # This prevents division by zero warning
    if label_sum == 0:
        return np.nan
    
    sorted_indices = np.argsort(scores)
    sorted_labels = labels.iloc[sorted_indices].values
    cumsum_labels = np.cumsum(sorted_labels)
    
    gini = 1 - 2 * np.sum(cumsum_labels) / (n * label_sum)
    return gini


def calculate_periodic_gini_prod_ver_trench_dimfact(df, score_column, label_column, namecolumn, 
                                        data_selection_column=None,
                                        model_version_column=None, trench_column=None, 
                                        loan_type_column=None, loan_product_type_column=None,
                                        ostype_column=None,
                                        account_id_column=None):
    """
    Calculate periodic Gini coefficients and return Power BI-friendly long format
    with fact and dimension tables.
    
    Returns:
    - fact_table: Long format with one row per segment per period
    - dimension_table: Unique segment combinations for filtering
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    data_selection_column: (optional) name of column for data selection (Test/Train)
    model_version_column: (optional) name of column for model version
    trench_column: (optional) name of column for trench category
    loan_type_column: (optional) name of loan type column
    loan_product_type_column: (optional) name of loan product type column
    ostype_column: (optional) name of column for OS type
    account_id_column: (optional) name of column for distinct account IDs
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    optional_columns = {
        'data_selection': data_selection_column,
        'model_version': model_version_column,
        'trench': trench_column,
        'loan_type': loan_type_column,
        'loan_product_type': loan_product_type_column,
        'ostype': ostype_column,
        'account_id': account_id_column
    }
    
    for col_name, col in optional_columns.items():
        if col and col not in df.columns:
            raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process
    datasets_to_process = [('Overall', df, {})]
    
    # Create list of available segment columns
    segment_columns = []
    if data_selection_column:
        segment_columns.append(('DataSelection', data_selection_column))
    if model_version_column:
        segment_columns.append(('ModelVersion', model_version_column))
    if trench_column:
        segment_columns.append(('Trench', trench_column))
    if loan_type_column:
        segment_columns.append(('LoanType', loan_type_column))
    if loan_product_type_column:
        segment_columns.append(('ProductType', loan_product_type_column))
    if ostype_column:
        segment_columns.append(('OSType', ostype_column))
    
    # Generate all possible combinations of segment columns
    for r in range(1, len(segment_columns) + 1):
        for combo in combinations(segment_columns, r):
            def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
                if current_filter is None:
                    current_filter = {}
                
                if index >= len(segment_columns):
                    filtered_df = df
                    for col, val in current_filter.items():
                        filtered_df = filtered_df[filtered_df[col] == val]
                    
                    if len(filtered_df) > 0:
                        yield (current_name.strip('_'), filtered_df, current_filter.copy())
                    return
                
                seg_name, seg_col = segment_columns[index]
                for seg_value in sorted(df[seg_col].dropna().unique()):
                    new_filter = current_filter.copy()
                    new_filter[seg_col] = seg_value
                    new_name = current_name + f'{seg_name}_{seg_value}_'
                    
                    yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
            
            for combo_name, combo_df, combo_metadata in generate_combinations(df, list(combo)):
                datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df, metadata in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini_value')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        
        # Add distinct account count for weekly
        if account_id_column:
            weekly_account_counts = dataset_df_copy.groupby('week')[account_id_column].nunique().reset_index()
            weekly_account_counts.columns = ['week', 'distinct_accounts']
            weekly_gini = weekly_gini.merge(weekly_account_counts, on='week', how='left')
        else:
            weekly_gini['distinct_accounts'] = None
        
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini_value', 'period', 'distinct_accounts']]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini_value')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        
        # Add distinct account count for monthly
        if account_id_column:
            monthly_account_counts = dataset_df_copy.groupby('month')[account_id_column].nunique().reset_index()
            monthly_account_counts.columns = ['month', 'distinct_accounts']
            monthly_gini = monthly_gini.merge(monthly_account_counts, on='month', how='left')
        else:
            monthly_gini['distinct_accounts'] = None
        
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini_value', 'period', 'distinct_accounts']]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['bad_rate'] = namecolumn
        gini_results['segment_type'] = dataset_name
        
        # Add individual segment components
        gini_results['data_selection'] = metadata.get(data_selection_column, None) if data_selection_column else None
        gini_results['model_version'] = metadata.get(model_version_column, None) if model_version_column else None
        gini_results['trench_category'] = metadata.get(trench_column, None) if trench_column else None
        gini_results['loan_type'] = metadata.get(loan_type_column, None) if loan_type_column else None
        gini_results['loan_product_type'] = metadata.get(loan_product_type_column, None) if loan_product_type_column else None
        gini_results['ostype'] = metadata.get(ostype_column, None) if ostype_column else None
        
        all_results.append(gini_results)
    
    # Combine all results
    fact_table = pd.concat(all_results, ignore_index=True)
    
    # Create dimension table (unique segment combinations for filtering)
    dimension_table = fact_table[['Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 
                                   'trench_category', 'loan_type', 'loan_product_type', 'ostype']].drop_duplicates().reset_index(drop=True)
    dimension_table['segment_id'] = range(len(dimension_table))
    
    # Add segment_id to fact table
    fact_table = fact_table.merge(dimension_table[['segment_id', 'Model_Name', 'bad_rate', 'segment_type', 
                                                     'data_selection', 'model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']], 
                                  on=['Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 
                                      'trench_category', 'loan_type', 'loan_product_type', 'ostype'], 
                                  how='left')
    
    # Reorder columns in fact table
    fact_table = fact_table[['segment_id', 'start_date', 'end_date', 'period', 'gini_value', 'distinct_accounts',
                             'Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 'trench_category', 
                             'loan_type', 'loan_product_type', 'ostype']]
    
    # Reorder columns in dimension table
    dimension_table = dimension_table[['segment_id', 'Model_Name', 'bad_rate', 'segment_type', 'data_selection', 
                                        'model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']]
    
    return fact_table, dimension_table


# Usage:
# fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     data_selection_column='Data_selection',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     loan_type_column='loan_type',
#     loan_product_type_column='loan_product_type',
#     ostype_column='osType',
#     account_id_column='digitalLoanAccountId'
# )
# 
# # In Power BI:
# # 1. Import fact_table and dimension_table
# # 2. Create relationship: dimension_table[segment_id] -> fact_table[segment_id]
# # 3. Use dimension table columns as filters (including ostype)
# # 4. Create DAX measures:
# #    - Gini Measure = AVERAGE(fact_table[gini_value])
# #    - Account Count = SUM(fact_table[distinct_accounts])
# # 5. Use start_date, end_date, period for time-based analysis

# update_tables

In [4]:

def update_tables(fact_table: pd.DataFrame, dimension_table: pd.DataFrame, model_name: str, product: str) -> tuple:
    """
    Updates fact_table and dimension_table:
    - Sets 'Model_display_name' to the given model_name
    - Replaces NaN values in specified columns with 'Overall'
    
    Returns:
        Updated fact_table and dimension_table as a tuple
    """
    # Columns where missing values should be replaced
    cols_to_replace = ['model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']
    
    # Update fact_table
    fact_table['Model_display_name'] = model_name
    fact_table['Product_Category'] = product
    fact_table[cols_to_replace] = fact_table[cols_to_replace].fillna('Overall')
    
    # Update dimension_table
    dimension_table['Model_display_name'] = model_name
    dimension_table['Product_Category'] = product
    dimension_table[cols_to_replace] = dimension_table[cols_to_replace].fillna('Overall')
    
    return fact_table, dimension_table


## alpha_stack_model_sil

### FPD0

#### Test

In [5]:
sq = """ 
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
     deviceOs osType,
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
      coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and flg_mature_fpd0 = 1
  )
  select *  from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (84962, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3777875,00d9749b-a742-4107-927f-19e3f7a29a25,60837778750011,alpha_stack_model_sil,0.1041017529157005,2025-10-29 16:46:34,2025-10-29,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
1,3734637,00f5eb46-dde2-47da-a884-74dcf0cb43b6,60837346370017,alpha_stack_model_sil,0.05457969882133833,2025-10-10 10:03:34,2025-10-10,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
2,3651229,01472cb7-83a9-45c9-b8cf-b9d172b6f9ad,60836512290017,alpha_stack_model_sil,0.1211813970579765,2025-08-30 10:32:01,2025-08-30,2025-08,Prod,1,1,SIL-Instore,v1,Trench 2,Mall,android
3,3729769,0161d70a-0028-4611-9f1c-0ad612f53522,60837297690013,alpha_stack_model_sil,0.08998031576230323,2025-10-07 16:39:03,2025-10-07,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
4,3716902,0225b2b0-5e1c-4e73-a6c8-79acfc67e45a,60837169020013,alpha_stack_model_sil,0.030969871066460892,2025-10-01 15:07:46,2025-10-01,2025-10,Prod,0,1,SIL ZERO,v1,Trench 2,Appliance,ios


In [6]:
df1 = dfd.copy()

#### Train

In [7]:
sq = """ 
  with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    Data_selection,
    deviceOs osType,
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
    deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and flg_mature_fpd0 = 1
  )
  select * from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (297167, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2265163,00596296-c050-44a0-b218-741fc108cba6,60822651630016,Alpha - StackingModel,0.156434,2023-10-08 17:25:15,2023-10-08,2023-10,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
1,2260578,0110811b-018f-494a-839d-d07d376ba9c9,60822605780015,Alpha - StackingModel,0.068144,2023-10-05 10:17:32,2023-10-05,2023-10,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
2,2225981,054ff623-eb19-4033-a09d-e3da32881c51,60822259810012,Alpha - StackingModel,0.091714,2023-09-08 18:11:23,2023-09-08,2023-09,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
3,2406213,08d5ddb2-cd31-4eb3-afc9-f08475ca448a,60824062130015,Alpha - StackingModel,0.050818,2024-02-20 16:57:37,2024-02-20,2024-02,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
4,2274175,0b169735-c257-4964-b459-5bd40596714f,60822741750011,Alpha - StackingModel,0.030425,2023-10-16 11:23:33,2023-10-16,2023-10,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android


In [8]:
df2 = dfd.copy()

#### Concatenate both Test and Train Datasets

In [9]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (341087, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3777875,00d9749b-a742-4107-927f-19e3f7a29a25,60837778750011,alpha_stack_model_sil,0.1041017529157005,2025-10-29 16:46:34,2025-10-29,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
1,3809657,0682bf5a-7a29-437d-81b1-c9db7604861b,60838096570012,alpha_stack_model_sil,0.09595339602984075,2025-11-14 13:36:25,2025-11-14,2025-11,Prod,1,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3756414,06e87962-2e6e-4597-82e6-be62cc0cb012,60837564140012,alpha_stack_model_sil,0.07412556196978269,2025-10-20 13:39:22,2025-10-20,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3788492,0870b2d5-5ea0-46b4-9742-29fb91931432,60837884920019,alpha_stack_model_sil,0.1535351460500835,2025-11-03 11:16:35,2025-11-03,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
4,3757092,0eecd98a-2246-43aa-8fe5-3254c7272827,60837570920018,alpha_stack_model_sil,0.05481600187976677,2025-10-20 17:00:03,2025-10-20,2025-10,Prod,0,1,SIL ZERO,v1,Trench 2,Appliance,android


#### Making Sure the Score is Numeric

In [10]:
df_concat['Sil_Alpha_Stack_score'] = pd.to_numeric(df_concat['Sil_Alpha_Stack_score'], errors='coerce')

#### Calculating the Gini

In [11]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'Sil_Alpha_Stack_score', 
    'deffpd0', 
    'FPD0',
    data_selection_column='Data_selection',  # Add this
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    ostype_column='osType', 
    account_id_column='digitalLoanAccountId'
)

#### Updating Fact and Dimension Table

In [12]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='alpha_stack_model_sil', product='SIL')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (79787, 17)
The shape of the dimension table is:	 (1946, 12)


#### Assigning the Table Name

In [13]:
facttable_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
dimtable_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"

#### Creating the table

In [14]:
# Upload to BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=5eec461a-7509-432d-a9c2-30dd001e74c6>

In [15]:
# Upload to BigQuery

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=379ef7ba-ceba-450b-899b-223429c3c4dc>

### FPD10

#### Test

In [16]:
sq = """ 
with modelname as 
  ( SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
   case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    deviceOs osType,
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
       coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fpd10 = 1
  )
  select *  from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (76043, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3691031,00dbe525-91fd-4606-8a6c-d7a398e13dd6,60836910310015,alpha_stack_model_sil,0.11223001857868516,2025-09-18 15:26:32,2025-09-18,2025-09,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,ios
1,3519122,01f5699b-6366-4150-9ab4-e73711c99116,60835191220013,alpha_stack_model_sil,0.05879045983873004,2025-06-25 13:46:23,2025-06-25,2025-06,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3793747,0201d55d-7dfc-4569-960c-db240da79521,60837937470018,alpha_stack_model_sil,0.11699688395045427,2025-11-05 19:46:13,2025-11-05,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3536238,0255eab7-d5fb-430c-ad27-cf27d48ddab8,60835362380012,alpha_stack_model_sil,0.04682858139172931,2025-07-03 18:39:31,2025-07-03,2025-07,Prod,0,1,SIL ZERO,v1,Trench 2,Appliance,android
4,3573745,035b4c9f-64d6-4c86-a826-5e3d4a9b3f52,60835737450014,alpha_stack_model_sil,0.050193204410967994,2025-07-23 14:31:35,2025-07-23,2025-07,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [17]:
df1 = dfd.copy()

#### Train

In [18]:
sq = """ 
  with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    Data_selection,
    deviceOs osType,
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
    del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fpd10 = 1
  )
  select * from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (297167, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2237980,008a74eb-d9e9-4150-94ce-39c7d1333dfd,60822379800014,Alpha - StackingModel,0.065810,2023-09-18 08:21:05,2023-09-18,2023-09,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
1,2211151,02398d7b-10da-41d8-9a50-d0a63f5e8ce5,60822111510011,Alpha - StackingModel,0.039318,2023-08-28 13:57:11,2023-08-28,2023-08,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
2,2287396,03c7efd5-f8f3-4345-978b-b856dee1bef4,60822873960019,Alpha - StackingModel,0.125891,2023-10-29 10:59:55,2023-10-29,2023-10,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
3,2206993,0d0ccea1-7fe6-493b-acf7-acbed630b0c3,60822069930011,Alpha - StackingModel,0.111872,2023-08-25 16:43:54,2023-08-25,2023-08,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,2250600,0f9be1d9-2bdb-4bda-847d-528a0d5c729b,60822506000011,Alpha - StackingModel,0.071110,2023-09-27 19:24:53,2023-09-27,2023-09,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [19]:
df2 = dfd.copy()

#### Concatenate both Test and Train Dataset

In [20]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (332168, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3793747,0201d55d-7dfc-4569-960c-db240da79521,60837937470018,alpha_stack_model_sil,0.11699688395045427,2025-11-05 19:46:13,2025-11-05,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
1,3806855,06df84cd-e535-4fc9-b05e-105cdf6f0f2c,60838068550014,alpha_stack_model_sil,0.12440023894794547,2025-11-12 16:05:44,2025-11-12,2025-11,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,ios
2,3923944,0eea2cfb-a1f5-4c3d-9adc-49932ac913ec,60839239440018,alpha_stack_model_sil,0.5216864789772874,2025-12-22 14:57:59,2025-12-22,2025-12,Prod,0,1,SIL Competitor,v2,Trench 1,Appliance,android
3,3793247,143d3ba6-353b-4dc0-bef0-bf62e2814b82,60837932470013,alpha_stack_model_sil,0.13104798933263798,2025-11-05 16:19:58,2025-11-05,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,3283691,1c3ba527-f4b2-4823-8994-cc96fb82da32,60832836910014,alpha_stack_model_sil,0.07559446023989827,2025-11-03 14:28:28,2025-11-03,2025-11,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android


#### Making Sure the Score is Numeric

In [21]:
df_concat['Sil_Alpha_Stack_score'] = pd.to_numeric(df_concat['Sil_Alpha_Stack_score'], errors='coerce')

#### Calculating the Gini

In [22]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'Sil_Alpha_Stack_score', 
    'deffpd10', 
    'FPD10',
    data_selection_column='Data_selection',  # Add this
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    ostype_column='osType',
    account_id_column='digitalLoanAccountId'
)

#### Updating Fact and Dimension Table

In [23]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='alpha_stack_model_sil', product='SIL')

In [24]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b7b5960d-bd75-4a6e-b7ea-ee51821a5128>

In [25]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e1dae5af-3f4f-4d89-8902-476d789211a6>

### FPD30

#### Test

In [26]:
sq = """ 
with modelname as 
  (SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
      deviceOs osType,
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
       coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fpd30 = 1
  )
  select *  from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (62597, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3646410,00445a33-818d-4ecf-952a-7e01f5de6cfc,60836464100017,alpha_stack_model_sil,0.13265001006972307,2025-08-27 15:39:10,2025-08-27,2025-08,Prod,1,1,SIL-Instore,v1,Trench 2,Mall,ios
1,3544172,00c019bb-b916-4f6b-8454-b78ba9fb1694,60835441720018,alpha_stack_model_sil,0.07331811953601028,2025-07-07 18:23:53,2025-07-07,2025-07,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
2,3745976,03d31bca-b8d4-4667-836f-d40c3b52bc60,60837459760012,alpha_stack_model_sil,0.03598528504763254,2025-10-15 14:29:03,2025-10-15,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
3,3562258,0404c5f6-08e3-4c6b-b724-0ed4fc631a1d,60835622580015,alpha_stack_model_sil,0.022187703978644734,2025-07-17 14:47:17,2025-07-17,2025-07,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
4,3603191,052a1894-c595-49aa-b1ac-9c515add99ed,60836031910013,alpha_stack_model_sil,0.09966637615456263,2025-08-06 14:53:42,2025-08-06,2025-08,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android


In [27]:
df1 = dfd.copy()

#### Train

In [28]:
sq = """ 
  with modelname as 
  (
  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    Data_selection,
       deviceOs osType,
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
    del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fpd30 = 1
  )
  select * from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (297167, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2405219,03cb7271-1688-4733-9300-7cf8ff3c6b5a,60824052190017,Alpha - StackingModel,0.142676,2024-02-19 17:38:36,2024-02-19,2024-02,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
1,2234185,04e8e400-9944-461a-a344-0fad9a951ecc,60822341850013,Alpha - StackingModel,0.071070,2023-09-15 11:11:49,2023-09-15,2023-09,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,2287489,1381b71b-8577-4ee1-b0d1-257099c3008f,60822874890013,Alpha - StackingModel,0.043613,2023-10-29 11:48:52,2023-10-29,2023-10,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,2033375,157e3b15-f091-47df-ad03-2ab71a301b9a,60820333750011,Alpha - StackingModel,0.025256,2023-05-07 16:39:21,2023-05-07,2023-05,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,2064911,16a15080-ee7a-4584-b2d9-73fe8f459fce,60820649110012,Alpha - StackingModel,0.109112,2023-05-28 10:40:35,2023-05-28,2023-05,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [29]:
df2 = dfd.copy()

#### Concatenate both Test and Train Datasets

In [30]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (318722, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3745976,03d31bca-b8d4-4667-836f-d40c3b52bc60,60837459760012,alpha_stack_model_sil,0.03598528504763254,2025-10-15 14:29:03,2025-10-15,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
1,3790814,09106d4c-8534-4571-bee8-7924de5bdc4c,60837908140011,alpha_stack_model_sil,0.10952524320000089,2025-11-04 15:55:27,2025-11-04,2025-11,Prod,1,1,SIL Competitor,v1,Trench 2,Mall,android
2,3801546,09775a40-eaaa-41ab-ae71-718f172c71c8,60838015460014,alpha_stack_model_sil,0.05516536894933103,2025-11-09 17:56:10,2025-11-09,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3760269,0cc37d7f-12f4-49be-8440-dcc3d9ed3888,60837602690012,alpha_stack_model_sil,0.04195750088589265,2025-10-22 11:49:32,2025-10-22,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
4,3833337,121240ec-018d-4f50-940e-bc81be68a046,60838333370011,alpha_stack_model_sil,0.03315672794230608,2025-11-24 19:08:08,2025-11-24,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android


#### Making Sure the Score is Numeric

In [31]:
df_concat['Sil_Alpha_Stack_score'] = pd.to_numeric(df_concat['Sil_Alpha_Stack_score'], errors='coerce')

#### Calculating the Gini

In [32]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'Sil_Alpha_Stack_score', 
    'deffpd30', 
    'FPD30',
    data_selection_column='Data_selection',  # Add this
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    ostype_column='osType',   
    account_id_column='digitalLoanAccountId'
)

#### Updating the Fact and Dimension Table

In [33]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='alpha_stack_model_sil', product='SIL')

#### Inserting the data into Fact and Dimension tables

In [34]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2fd182e9-ba52-4b3a-8102-fb0c26b9fcf9>

In [35]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c7fb3fe2-9e17-4c88-822a-4d56da9691d8>

### FSPD30

#### Test

In [36]:
sq = """ 
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    deviceOs osType,
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
   coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fspd_30 = 1
  )
  select *  from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (47478, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3571935,008adc78-8b72-4e2b-82d3-ff3b1fb3696b,60835719350011,alpha_stack_model_sil,0.04771735908456503,2025-07-22 14:40:37,2025-07-22,2025-07,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
1,3676064,0308fa1c-6020-4e3e-8755-125d70cf89c2,60836760640014,alpha_stack_model_sil,0.10866212174791978,2025-09-10 14:43:01,2025-09-10,2025-09,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3444696,08d02c3f-e6ef-412b-915a-ef072aba31e9,60834446960015,alpha_stack_model_sil,0.03541934981328111,2025-05-18 18:35:50,2025-05-18,2025-05,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3647556,13e28537-f8c1-4181-9d91-a4c48f9c8c94,60836475560018,alpha_stack_model_sil,0.047439400320832,2025-08-28 11:21:49,2025-08-28,2025-08,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
4,3493252,17fd3c67-8c51-475e-bc61-59da73578df4,60834932520012,alpha_stack_model_sil,0.12540181837654646,2025-06-12 16:46:31,2025-06-12,2025-06,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,android


In [37]:
df1 = dfd.copy()

#### Train

In [38]:
sq = """ 
  with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
   case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    Data_selection,
        deviceOs osType,
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
    del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fspd_30 = 1
  )
  select * from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (297167, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2252518,0a04ef85-5ae4-4e5f-97e2-2c152ac662d8,60822525180016,Alpha - StackingModel,0.077230,2023-09-29 14:44:11,2023-09-30,2023-09,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
1,2241093,0fe73c23-9b6d-479c-a2c7-fdbeca60759e,60822410930013,Alpha - StackingModel,0.030820,2023-09-20 15:16:07,2023-09-20,2023-09,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,2228542,1227a66a-f745-457d-b16e-f34549c4076c,60822285420015,Alpha - StackingModel,0.333309,2023-09-10 18:13:49,2023-09-10,2023-09,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
3,2275589,19b2e67c-0e33-43c2-b066-597555700df6,60822755890013,Alpha - StackingModel,0.239525,2023-10-17 14:29:59,2023-10-17,2023-10,Pre_Train,1,1,SIL-Instore,v1,Trench 2,Appliance,android
4,1949396,1bc1c653-ad53-4b3d-9dd1-24326e3c617b,60819493960012,Alpha - StackingModel,0.269651,2023-05-07 10:58:27,2023-05-07,2023-05,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [39]:
df2 = dfd.copy()

#### Concatenate both Test and Train Datasets

In [40]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (303603, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3784381,9e4266ef-a126-4918-a36a-926bc7964940,60837843810017,alpha_stack_model_sil,0.09630789111329685,2025-11-01 14:40:39,2025-11-01,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
1,3784959,d4747f88-7acd-45bc-9664-7ca202fedd12,60837849590017,alpha_stack_model_sil,0.08397611408707599,2025-11-01 18:18:38,2025-11-01,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3784007,032a90a0-4e68-494c-a76d-f4c69ad1acbc,60837840070018,alpha_stack_model_sil,0.05879478860698301,2025-11-01 12:45:31,2025-11-01,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3784111,0fd1e919-a0c2-4923-8bd8-ef13c952e524,60837841110017,alpha_stack_model_sil,0.07456131688840281,2025-11-01 13:31:31,2025-11-01,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
4,3784727,1ba5c7bc-56d8-4d75-8d97-78cdb9ee66ad,60837847270013,alpha_stack_model_sil,0.05859976745919431,2025-11-01 16:24:04,2025-11-01,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,ios


#### Making sure the score column in numeric

In [41]:
df_concat['Sil_Alpha_Stack_score'] = pd.to_numeric(df_concat['Sil_Alpha_Stack_score'], errors='coerce')

#### Calculating the Gini

In [42]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'Sil_Alpha_Stack_score', 
    'deffspd30', 
    'FSPD30',
    data_selection_column='Data_selection',  # Add this
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    ostype_column='osType', 
    account_id_column='digitalLoanAccountId'
)

#### Updating the Fact and Dimension Table

In [43]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='alpha_stack_model_sil', product='SIL')

#### Inserting the Data into Fact and Dimension Table

In [44]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f7df9b71-afdd-485c-81c7-daf31989c668>

In [45]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=dd530485-81e6-479d-9b1c-ae04cde2a0a8>

### FSTPD30

#### Test

In [46]:
sq = """ 
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    deviceOs osType,
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fstpd_30 = 1
  )
  select *  from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (36932, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2592905,0b83ae84-c654-4dbd-848a-13203dcd1692,60825929050029,alpha_stack_model_sil,0.014214134020566479,2025-04-02 17:13:37,2025-04-02,2025-04,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,android
1,3490609,0f0ae94b-4052-489f-ac70-86dbc7cb57a8,60834906090019,alpha_stack_model_sil,0.08302672445290757,2025-06-11 12:15:29,2025-06-11,2025-06,Prod,1,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3489618,12cce38d-5497-47da-81b4-99ec8a744822,60834896180011,alpha_stack_model_sil,0.07199743369375584,2025-06-10 18:23:52,2025-06-10,2025-06,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3449263,1cac97c5-7934-4fe5-9154-47ff5ee40a96,60834492630014,alpha_stack_model_sil,0.024511495650289636,2025-05-21 11:14:58,2025-05-21,2025-05,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
4,3449473,26f229f5-fa30-403c-bcff-b612cc084163,60834494730012,alpha_stack_model_sil,0.03505873636476556,2025-05-21 13:00:56,2025-05-21,2025-05,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [47]:
df1 = dfd.copy()

#### Train

In [48]:
sq = """ 
  with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
   case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    Data_selection,
        deviceOs osType,
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
    del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fstpd_30 = 1
  )
  select * from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (292138, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2064940,01fdfb53-8bb0-4a8b-8360-4ec7a944872f,60820649400017,Alpha - StackingModel,0.074059,2023-05-28 10:57:29,2023-05-28,2023-05,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
1,2123681,0217f3ad-e0f0-42e6-a6cd-beb300c09727,60821236810019,Alpha - StackingModel,0.063021,2023-07-07 14:51:29,2023-07-07,2023-07,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
2,2286861,0402bb12-0857-49dc-ab27-bdcb258bc557,60822868610019,Alpha - StackingModel,0.024449,2023-10-28 16:29:14,2023-10-28,2023-10,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
3,2288542,0ae76998-8a11-43cc-8da0-c207ca1bd59f,60822885420015,Alpha - StackingModel,0.182048,2023-10-30 12:12:03,2023-10-30,2023-10,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,2208143,0fa50ec3-f3fc-4c06-b161-edc0063170f7,60822081430014,Alpha - StackingModel,0.018230,2023-08-26 14:35:22,2023-08-26,2023-08,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [49]:
df2 = dfd.copy()

#### Concatenate both Test and Train Datasets

In [50]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (292177, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3071132,b73e11f2-2886-45d8-b911-4208f166f6fc,60830711320013,alpha_stack_model_sil,0.010246952080656055,2025-03-28 15:32:24,2025-03-29,2025-03,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,android
1,3431236,3b19bc86-07f3-4f35-839d-ebd5c9c9f7f0,60834312360014,alpha_stack_model_sil,0.14732277538919222,2025-05-11 13:56:41,2025-05-11,2025-05,Prod,1,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3482558,e875982d-ea4b-492e-a5ef-a5304b679d4e,60834825580018,alpha_stack_model_sil,0.1377673123601746,2025-06-07 09:20:45,2025-06-11,2025-06,Prod,1,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3358816,b22207e8-7643-41e8-a199-9842bde1ae8d,60833588160011,alpha_stack_model_sil,0.010246952080656055,2025-04-04 12:31:28,2025-04-04,2025-04,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,3495127,9dd8b6fa-0a40-4c6f-b888-cc3108a18953,60834951270016,alpha_stack_model_sil,0.10882048423582898,2025-06-13 16:53:08,2025-06-13,2025-06,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android


#### Making sure the score column is numeric

In [51]:
df_concat['Sil_Alpha_Stack_score'] = pd.to_numeric(df_concat['Sil_Alpha_Stack_score'], errors='coerce')

#### Calculating the Gini

In [52]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'Sil_Alpha_Stack_score', 
    'deffstpd30', 
    'FSTPD30',
    data_selection_column='Data_selection',  # Add this
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
     ostype_column='osType', 
    account_id_column='digitalLoanAccountId'
)

#### Updating the Fact and Dimension Tables

In [53]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='alpha_stack_model_sil', product='SIL')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (70239, 17)
The shape of the dimension table is:	 (1687, 12)


In [54]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=694fbcad-a8e4-4aba-908d-80d14d786537>

In [55]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=167add2d-ebaf-4aff-a068-8452d68906bd>